In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
!PIP_DISABLE_PIP_VERSION_CHECK=1 pip3 install kfp==0.1.36 --user --quiet

In [ ]:
FIRSTNAME_LASTNAME = 'firstname_lastname'
EXPERIMENT_NAME = 'Hello world! - metrics and visualizations ' + FIRSTNAME_LASTNAME

# Visualize Results in the Pipelines UI
https://www.kubeflow.org/docs/pipelines/sdk/output-viewer/

In [ ]:
import kfp.components as comp

In [ ]:
#Define a Python function
from typing import NamedTuple
def view_metric(a: float, b: float) -> NamedTuple('MyMetricOutput', [('value', float), ('mlpipeline_metrics', 'Metrics')]):
    
    from tensorflow.python.lib.io import file_io
    import json

    # Exports two sample metrics:
    metrics = {
      'metrics': [{
          'name': 'metric-a',
          'numberValue':  float(a),
        },{
          'name': 'metric-b',
          'numberValue':  float(b),
        }]}

    from collections import namedtuple
    divmod_output = namedtuple('MyDivmodOutput', ['value', 'mlpipeline_metrics'])
    return divmod_output(a, json.dumps(metrics))

In [ ]:
view_metric_op = comp.func_to_container_op(view_metric, base_image='tensorflow/tensorflow:1.11.0-py3')

#### Define the pipeline
Pipeline function has to be decorated with the `@dsl.pipeline` decorator

In [ ]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Visualize results',
   description='A toy pipeline that demonstrates how to visualize results in the Pipelines UI'
)
def visualize(
   a='a',
   b='7'
):
    #building a one step pipeline to Visualize the results 
    view_metric_task = view_metric_op(a, b)
    

#### Compile the pipeline

In [ ]:
pipeline_func = visualize
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

In [ ]:
#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

In [ ]:
#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' ' + FIRSTNAME_LASTNAME + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)